In [8]:
from  bs4 import BeautifulSoup
import pandas as pd
import requests
import pprint 
pp = pprint.PrettyPrinter(indent=4)

news_url = "https://economictimes.indiatimes.com/markets/stocks/news"
response = requests.get(news_url)
soup = BeautifulSoup(response.text, 'html.parser')
each_story_divs = soup.find_all('div', class_='eachStory')
# storyPage=news_url+ each_story_divs[1].find('a')['href']
storyPages = [
    news_url + story_div.find('a')['href']
    for story_div in each_story_divs
    if story_div.find('a') and 'href' in story_div.find('a').attrs
]

data = []  # List to store extracted data

for i, storyPage in enumerate(storyPages, 1):
    try:
        response = requests.get(storyPage, timeout=10)
        response.raise_for_status()  # Raise an exception for 4xx/5xx errors
        
        soup = BeautifulSoup(response.text, 'html.parser')
        page_content = soup.find('div', class_='artText')

        if page_content:
            text_content = page_content.get_text(strip=True)
        else:
            text_content = "Content not found"

        relative_path = storyPage.replace(news_url, "")
        data.append({'Relative Path': relative_path, 'Content': text_content})
        print(f"✅ {i}/{len(storyPages)}: Scraped {relative_path}")

    except requests.exceptions.RequestException as e:
        print(f"❌ Failed to fetch {storyPage}: {e}")
        data.append({'URL': storyPage, 'Content': 'Request failed'})

# Convert to DataFrame
df = pd.DataFrame(data)

# Display DataFrame
print(df.head())

# Optionally, save data to CSV
df.to_csv("scraped_news.csv", index=False)


✅ 1/16: Scraped /markets/stocks/news/foreign-investors-snap-month-long-selling-streak-turn-positive-in-indian-equities/articleshow/117922810.cms
✅ 1/16: Scraped https://economictimes.indiatimes.com/markets/stocks/news/markets/stocks/news/foreign-investors-snap-month-long-selling-streak-turn-positive-in-indian-equities/articleshow/117922810.cms
✅ 2/16: Scraped /markets/stocks/news/ahead-of-market-10-things-that-will-decide-stock-market-action-on-wednesday/articleshow/117922469.cms
✅ 2/16: Scraped https://economictimes.indiatimes.com/markets/stocks/news/markets/stocks/news/ahead-of-market-10-things-that-will-decide-stock-market-action-on-wednesday/articleshow/117922469.cms
✅ 3/16: Scraped /markets/stocks/news/market-trading-guide-kotak-bank-graphite-india-are-among-4-stock-recommendations-for-wednesday/slideshow/117920181.cms
✅ 3/16: Scraped https://economictimes.indiatimes.com/markets/stocks/news/markets/stocks/news/market-trading-guide-kotak-bank-graphite-india-are-among-4-stock-recomm

In [46]:
import pprint 
text2=requests.get("https://json.bselivefeeds.indiatimes.com/marketband.json").text
pprint.pprint(text2)

('marketlivedata({"marketBandList":[{"sectionName":"S1","bandServiceMetas":[{"serviceName":"Nifty","serviceType":"INDEX","landingPageUrl":"https://economictimes.indiatimes.com/markets/indices/nifty-50","netChange":-26.25,"percentChange":-0.11,"dateTime":"01-February-2025 '
 '16:15:31","currentIndexValue":23482.15,"scripCode":"NSE '
 'Index","exchangeID":"50","seoName":"nifty-50","open":23528.6,"high":23632.45,"low":23318.3,"current":23482.15,"indexId":"2369","dateTimeUnix":1738406731000},{"serviceName":"Sensex","serviceType":"INDEX","landingPageUrl":"https://economictimes.indiatimes.com/markets/indices/bse-sensex","netChange":5.39,"percentChange":0.0,"dateTime":"01-February-2025 '
 '16:15:59","currentIndexValue":77505.96,"scripCode":"Sensex","exchangeID":"47","seoName":"bse-sensex","open":77637.01,"high":77899.05,"low":77006.47,"current":77505.96,"indexId":"2365","dateTimeUnix":1738406759000}]},{"sectionName":"S2","bandServiceMetas":[{"serviceName":"Gainer","serviceType":"LIVE_STATS","

<H3>Model importing from hugging face finbert</H3>

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Enable GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and model only once
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert").to(device)
model.eval()  # Set model to evaluation mode

# Function for fast sentiment prediction
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
    
    # Use inference mode for faster processing
    with torch.inference_mode():
        outputs = model(**inputs)
    
    # Compute probabilities
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    
    # Sentiment labels
    sentiment_labels = [ "positive","negative","neutral", ]
    predicted_sentiment = sentiment_labels[torch.argmax(probs)]

    return predicted_sentiment, probs.tolist()

# Example usage
text = PageContent
sentiment, probabilities = predict_sentiment(text)

# Display results
print(f"Text: {text}")
print(f"Predicted Sentiment: {sentiment}")
print(f"Probabilities: {probabilities}")


Text: transformers: Rise of the Beasts box office collection: Transformers movie collects $8.8 million in previews
Predicted Sentiment: positive
Probabilities: [[0.5008206367492676, 0.1127873882651329, 0.3863920271396637]]
